**Expectation Maximization** <br>

We want to maximize the parameters $\theta$ that maximize the log likelihood given some observed data $x$.

\begin{align}
\argmax_{\theta} \mathrm{l}(\theta)=\argmax_{\theta} \log p(x\mid\theta)
\end{align}

if the full data however is given by $(z,x)$ and $z$ is a hidden probabilistic variable that follows a distribution $p(z)$ which we seek to find we can write:

\begin{align}
\log p(x\mid\theta)=\log \sum_{z\in Z} p(x,z\mid\theta)
\end{align}

Optimizing this expression is diffficuls due to the order of the summation and the logarithm. For such a scenario an analytic solution is not feasible and we have to resort to a technique called expectation maximization. We can introduce an arbitrary probability density $q(z)$ over the hidden variable and use Jensen's inqeuality to get:

\begin{align}
\log p(x\mid\theta) &= \log \sum_{z\in Z} p(x,z\mid\theta) \\
&= \log \sum_{z\in Z} q(z) \frac{p(x,z\mid\theta)}{q(z)} \\
&= \log \mathrm{E}_{z \sim q(z)}  \left[ \frac{p(x,z\mid\theta)}{q(z)} \right] \\
&\underset{\mathrm{Jensen's}}{\geq} \mathrm{E}_{z \sim q(z)}  \left[ \log \frac{p(x,z\mid\theta)}{q(z)} \right] \\
&=\underbrace{\mathrm{E}_{z \sim q(z)}  \log p(x,z\mid\theta)}_{\equiv Q(\theta)} -  \mathrm{E}_{z \sim q(z)} \log q(z) \\
&\equiv \mathrm{ELBO}
\end{align}

Since $\log q(z)$ is independent of $\theta$ we can optimize the the log-likelihood by optimizing the lower bound $Q(\theta)$. In order to optimize the lower bound we need to compute the expectation using the distribution $q(z)$ which is unknwon. However by rearraing the expression for the $\mathrm{ELBO}$ we find:

\begin{align}
\mathrm{ELBO} &=\mathrm{E}_{z \sim q(z)}  \left[ \log \frac{p(x,z\mid\theta)}{q(z)} \right] \\
&=\mathrm{E}_{z \sim q(z)}  \left[ \log \frac{p(x,z\mid\theta)p(z\mid x,\theta)}{q(z)p(z\mid x,\theta)} \right] \\
&=\mathrm{E}_{z \sim q(z)}  \left[ \log \frac{p(x\mid\theta)p(z\mid x,\theta) p(z\mid x,\theta)}{q(z)p(z\mid x,\theta)} \right] \\
&=\mathrm{E}_{z \sim q(z)}  \left[ \log \frac{p(x\mid\theta) p(z\mid x,\theta)}{q(z)} \right] \\
&= \log p(x\mid\theta) - \mathrm{KL}(q(z) \mid\mid p(z\mid x,\theta))
\end{align}
In the last step we used that $\log p(x\mid\theta)$ is independent of $z$. We can now see that the optimal $q(z)$ is given by $p(z\mid x,\theta)$ since in this case the Kullback-Leibler Divergence is minimized and therefore the $\mathrm{ELBO}$ maximized. <br>
We now have two objectives: (1) optimizing the lower bound $Q(\theta)$. (2) choosing $q(z)=p(z\mid x,\theta)$. The expectation maximization algorithm works by repeating those two steps until convergence:
* E-Step: compute $p(z\mid x,\theta)$ using fixed $\theta$ in order to obtain $q(z)$.
* M-Step: update $\theta$ <- $\argmax_{\theta} Q(\theta)$. <br>

The initial $\theta$ can be picked at random.

***Example Sketch: Gaussian Mixture Model (GMM)*** <br>

In the GMM the probabilty of observing event $X_i=x$ is given by a mixture of $k=1,...,n$ gaussian distributions $P(X_i=x\mid Z_i=k)$ that are weighted by a factor $\pi_i$ describing the probability that $x_i$ was drawn from i'th mixture component. 

\begin{align}
P(X_i=x)=\sum_{k=1}^{n} \pi_{k}P(X_i=x\mid Z_i=k) \quad \mathrm{with} \quad \sum_i \pi_i =1
\end{align}

Since we are dealing with Gaussian Distributions, $P(X_i=x\mid Z_i=k)=N(\sigma^{2}_{k}, \mu_k)$. The log-Likelihood is given by:

\begin{align}
l(\theta)&=\log \prod_{i=1}^n\sum_{k=1}^{n} \pi_{k}N(\sigma^{2}_{k}, \mu_k) \\
&= \sum_{i=1}^n \log \sum_{k=1}^{n} \pi_{k}N(\sigma^{2}_{k}, \mu_k)
\end{align}

If we knew from which of the Gaussian a given datapoint was sampled we could go ahead a find a analytic solution to this problem since it would simplify to a MLE of a single Gaussian Distribution with $\pi_i=1/N_i$ with $N_i$ being the number of samples from the i'th Gaussian. Sor in the case of GMM our latent variable $z$ is the membership. In order to find the solution to this Problem we can apply EM:

* E-step: compute $p(z\mid x,\theta)$ for a single datapoint and fixed parameters $\theta$:

\begin{align}
p(Z=z_k\mid X=x_k,\theta_k)&=p(Z=z_k\mid X=x_k,\sigma^{2}_{k}, \mu_k) \\
&=\frac{\pi_k N(\sigma^{2}_{k}, \mu_k)}{\sum_{k=1}^{n} \pi_k N(\sigma^{2}_{k}, \mu_k)} \\
&\equiv \gamma(k)
\end{align}

\begin{align}
\Rightarrow p(Z\mid X,\theta)=\prod_{i=1}^k \frac{\pi_k N(\sigma^{2}_{k}, \mu_k)}{\sum_{k=1}^{n} \pi_k N(\sigma^{2}_{k}, \mu_k)} \\
\end{align}


* M-step: compute $\mathrm{E}_{z \sim q(z)} \log p(x,z\mid\theta)$ by setting $p(Z\mid X,\theta)=q(z)$. Since we computed it for fixed parameters $\theta$, $q(z)$ only depends upon $z$. We can then compute the derivatives with respect to the paremters and update them. <br>



**Repameterization Trick** <br>

Say we want to compute the gradient of the expectation of a function $f(x)$ with $x\sim P(\sigma)$ with respect to $\sigma$. In the first general anaylsis we don't impose the contrain that $f$ is independent of $\sigma$. Taking the gradient Expecation  yields:

\begin{align}
\nabla_{\sigma} \mathrm{E}_{x\sim P(\sigma)}[f(x)] &= \nabla_{\sigma} \int f(x)N(x\mid \sigma,1)dx \\
&= \int f(x) \nabla_{\sigma} P(x\mid \sigma)dx + \int P(x\mid \sigma) \nabla_{\sigma}f(x) dx\\
&= \int f(x) P(x\mid \sigma) \nabla_{\sigma} log(P(x\mid \sigma))dx +\int P(x\mid \sigma) \nabla_{\sigma}f(x) dx \\
&= \mathrm{E}_{x\sim P(\sigma)}[f(x) \nabla_{\sigma} log(P(x\mid \sigma))] + \mathrm{E}_{x\sim P(\sigma)}[\nabla_{\sigma}f(x)]
\end{align}

The first term in this equation contains the gradient of the probability distribution which is often intractable. The repameterization trick is a way to circumvent this problem. We can rewrite the expectation as:

\begin{align}
\nabla_{\sigma} \mathrm{E}_{x\sim P(\sigma)}[f(x)] &= \nabla_{\sigma} \mathrm{E}_{\epsilon\sim P(\epsilon)}[f(g(\epsilon,\sigma))] \\
&= \nabla_{\sigma} \int f(g(\epsilon,\sigma))P(\epsilon)d\epsilon \\
&= \int f(g(\epsilon,\sigma)) \nabla_{\sigma} P(\epsilon)d\epsilon + \int P(\epsilon) \nabla_{\sigma}f(g(\epsilon,\sigma)) d\epsilon \\
&= \int f(g(\epsilon,\sigma)) \nabla_{\sigma} log(P(\epsilon))d\epsilon + \int P(\epsilon) \nabla_{\sigma}f(g(\epsilon,\sigma)) d\epsilon \\
&= \mathrm{E}_{\epsilon\sim P(\epsilon)}[f(g(\epsilon,\sigma)) \nabla_{\sigma} log(P(\epsilon))] + \mathrm{E}_{\epsilon\sim P(\epsilon)}[\nabla_{\sigma}f(g(\epsilon,\sigma))]
\end{align}
since the probability distribution $P(\epsilon)$ is independent of $\sigma$ we can now compute the gradient of the expectation by sampling from $P(\epsilon)$ and computing the gradient of $f$ with respect to $\sigma$.

\begin{align}
\nabla_{\sigma} \mathrm{E}_{x\sim P(\sigma)}[f(x)] 
&= \mathrm{E}_{\epsilon\sim P(\epsilon)}[\nabla_{\sigma}f(g(\epsilon,\sigma))]
& \approxeq \frac{1}{N} \sum_{i=1}^{N} \nabla_{\sigma}f(g(\epsilon_i,\sigma))
\end{align}

***Example Sketch: $f(x)=x^2$, $x \sim \mathrm{N}(\sigma, 1)$***<br>

We want to compute the gradient of the expectation of $f(x)=x^2$ with respect to $\sigma$ where $x\sim \mathrm{N}(\sigma, 1)$. We can rewrite the expectation:

\begin{align}
\nabla_{\sigma} \mathrm{E}_{x\sim P(\sigma)}[f(x)] &= \nabla_{\sigma}\int p(x)x^2dx \\
&= \nabla_{\sigma}\int N(x\mid \sigma,1)x^2dx \\
&= \int x^2 \nabla_{\sigma}N(x\mid \sigma,1)\\
&= \int x^2 N(x\mid \sigma,1) \nabla_{\sigma}log(N(x\mid \sigma,1))dx \\
&= \mathrm{E}_{x\sim P(\sigma)}[x^2 \nabla_{\sigma}log(N(x\mid \sigma,1))] \\
&= \mathrm{E}_{x\sim P(\sigma)}[x^2 \nabla_{\sigma}(-\frac{1}{2}(x-\sigma)^2)] \\
&= \mathrm{E}_{x\sim P(\sigma)}[x^2 \nabla_{\sigma}(-\frac{1}{2}(x^2-2x\sigma+\sigma^2))] \\
&= \mathrm{E}_{x\sim P(\sigma)}[x^2 \nabla_{\sigma}(-\frac{1}{2}x^2+x\sigma-\frac{1}{2}\sigma^2)] \\
&= \mathrm{E}_{x\sim P(\sigma)}[x^2 (-x+\sigma)] \\
\end{align}

By reparameterization using $\epsilon \sim \mathrm{N}(0,1)$ we obtain the following expression: $x=g(\epsilon,\sigma)=\sigma+\epsilon$. We can now use $\mathrm{E}_{\epsilon\sim P(\epsilon)}[\nabla_{\sigma}f(g(\epsilon,\sigma))]$ to compute the expectation, since the probability distribution $P(\epsilon)$ is independent of $\sigma$:
\begin{align}
\nabla_{\sigma} \mathrm{E}_{x\sim P(\sigma)}[f(x)] =& \mathrm{E}_{\epsilon\sim P(\epsilon)}[\nabla_{\sigma}f(g(\epsilon,\sigma))] \\
&= \mathrm{E}_{\epsilon\sim P(\epsilon)}[\nabla_{\sigma}(\sigma+\epsilon)^2] \\
&= \mathrm{E}_{\epsilon\sim P(\epsilon)}[2(\sigma+\epsilon)] \\
\end{align}
By running a small simulation we can see that we dramatically reduce the variance of the gradient by reparameterization. This leads in a real world scenario to a faster convergence of the optimization algorithm.





In [22]:
import numpy as np
import matplotlib.pyplot as plt
sigma = 1
mu = 0.5
epsilon = np.random.randn(10000)
x = mu + sigma * epsilon

In [23]:
var = np.var(x**2*(-x+mu))
var_reparameterized = np.var(2*(mu+epsilon))
print(f"Variance of the gradient: {var:.2f}")
print(f"Variance of the reparameterized gradient: {var_reparameterized:.2f}")

Variance of the gradient: 18.88
Variance of the reparameterized gradient: 4.03
